In [2]:
from flask import Flask, redirect, url_for,jsonify,request
from flask_cors import CORS, cross_origin
import pandas as pd
import pmdarima as pm
from pmdarima import model_selection, auto_arima
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from prophet import Prophet
import mysql.connector

In [3]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="Corona1234!",
  database="corona"
)
print(mydb)

In [4]:
app = Flask(__name__)
@app.route("/")
@cross_origin()
def home():
    City_Code=request.args.get('cityCode')
    City_Code=int(City_Code)
    mycursor = mydb.cursor()
    mycursor.execute("SELECT Date,Accumulated_Verified_Cases FROM city_status_vaccine_total WHERE City_Code=%s"%City_Code)
   
    myresult = mycursor.fetchall()
    df = pd.DataFrame(myresult,columns=["Date","Accumulated_Verified_Cases"])
    
    df['Date'] = pd.to_datetime(df['Date'])
    df['Date'] = df['Date'].dt.strftime("%d/%m/%Y")

    df_arima = df.set_index('Date')
    df_arima["avc_diff"] = df_arima.Accumulated_Verified_Cases.diff()
    df_arima.dropna(inplace=True)
    
    days4 = df_arima.tail(14)
    print(days4)

    days14Before = days4['avc_diff'].to_list()
    print(days14Before)

    for i in range(0, len(days14Before)):
        days14Before[i] = int(days14Before[i])
        
    print(days14Before)

    train, test = model_selection.train_test_split(df_arima['avc_diff'],train_size = 0.8)
    arima = pm.auto_arima(train,seasonal=True)
    forecast = arima.predict(n_periods=len(test)) 
    arima_rmse = np.sqrt(mean_squared_error(test, forecast))

    df_prophet = df_arima.copy()
    df_prophet.reset_index(inplace=True)
    df_prophet.rename(columns={'Date': 'ds','avc_diff': 'y'}, inplace=True)    
    
    train, test = model_selection.train_test_split(df_prophet,train_size = 0.8)
    m = Prophet() 
    m.fit(train)
    
    future = m.make_future_dataframe(periods=len(test)) 
    forecast = m    .predict(future)
    prophet_rmse = np.sqrt(mean_squared_error(test.y, forecast.yhat[-len(test):])) 
    if (arima_rmse>prophet_rmse):

        m = Prophet()
        m.fit(df_prophet) 
        future = m.make_future_dataframe(periods=14) 
        forecast = m.predict(future)
        future_val = forecast.yhat[-14:].to_list() 
    else:
        df_arima2 = df_arima[['avc_diff']]
        arima = pm.auto_arima(df_arima2) 
        future_val = list(arima.predict(n_periods=14))
 

    for i in range(0, len(future_val)):
        future_val[i] = int(future_val[i])
    days14Before.extend(future_val)
    print (days14Before)
    print (future_val)
    return jsonify(days14Before)

In [5]:
if __name__ =="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
